# Simulation Box

A `Box` defines the simulation cell: its shape, periodic boundary conditions, PBC, and the coordinate transforms you need to work safely with periodic systems.

You will mostly use `Box` for wrapping coordinates into the primary cell, `wrap`, Minimum-image distance vectors and distances, `diff`, `dist`, and Converting between absolute and fractional coordinates, `make_fractional`, `make_absolute`.

MolPy represents the box as a $3\times 3$ matrix whose **columns** are lattice vectors. From that matrix, MolPy can work with three common styles:
**FREE**: no boundaries / no periodicity, **ORTHOGONAL**: rectangular, diagonal matrix, and **TRICLINIC**: general cell with tilts, off-diagonal terms.


## 1. Create boxes

Use factory constructors for common cells, or pass a matrix when you already have lattice vectors. Keep your unit system consistent with the coordinates you will store, Å, nm, etc..

In [ ]:
import molpy as mp
import numpy as np

cubic = mp.Box.cubic(20.0)
ortho = mp.Box.orth([10.0, 20.0, 30.0])

# LAMMPS-style triclinic: lengths + tilts (xy, xz, yz)
tric = mp.Box.tric(lengths=[10.0, 12.0, 15.0], tilts=[1.0, 0.5, 0.2])

# Direct matrix construction (columns are lattice vectors)
matrix = np.array([[10.0, 1.0, 0.5], [0.0, 12.0, 0.2], [0.0, 0.0, 15.0]])
box_from_matrix = mp.Box(matrix=matrix)

{
    "cubic_style": str(cubic.style),
    "ortho_lengths": ortho.lengths.tolist(),
    "tric_tilts": tric.tilts.tolist(),
    "matrix": box_from_matrix.matrix,
}

## 2. Periodic boundary conditions

PBC controls whether coordinates wrap around the box on each axis. Common setups:
bulk system: periodic in `x/y/z` and slab/interface: periodic in `x/y`, non-periodic in `z`.


In [ ]:
import molpy as mp

bulk = mp.Box.cubic(10.0)
slab = mp.Box.orth([20.0, 20.0, 50.0], pbc=[True, True, False])

{
    "bulk_pbc": bulk.pbc.tolist(),
    "slab_pbc": slab.pbc.tolist(),
    "slab_periodic_z": slab.periodic_z,
}

## 3. Geometry and derived properties

`Box` exposes derived properties from its matrix representation. You’ll typically inspect:
`style`: FREE / ORTHOGONAL / TRICLINIC, `lengths`, `volume`, `origin`, `bounds`, and, triclinic only `tilts`, `angles`.


In [ ]:
import molpy as mp
import numpy as np

ortho = mp.Box.orth([10.0, 12.0, 15.0])
tric = mp.Box.tric(lengths=[10.0, 12.0, 15.0], tilts=[1.0, 0.5, 0.2])

{
    "ortho": {
        "style": str(ortho.style),
        "lengths": ortho.lengths.tolist(),
        "volume": float(ortho.volume),
        "origin": ortho.origin.tolist(),
        "bounds": ortho.bounds.tolist(),
        "matrix": ortho.matrix,
    },
    "tric": {
        "style": str(tric.style),
        "lengths": tric.lengths.tolist(),
        "tilts": tric.tilts.tolist(),
        "angles_deg": tric.angles.tolist(),
        "volume": float(tric.volume),
        "matrix": tric.matrix,
    },
}

## 4. Coordinate transforms and wrapping

Two safe patterns when working with PBC:
Convert to fractional coordinates, easy to reason about periodic images and Wrap positions into the primary cell before analysis/export.


In [ ]:
import numpy as np
import molpy as mp

box = mp.Box.cubic(10.0)

points = np.array([
    [12.0, -2.0, 5.0],
    [25.0, 8.0, -3.0],
], dtype=float)

wrapped = box.wrap(points)
images = box.get_images(points)
unwrapped = box.unwrap(wrapped, images)

absolute = np.array([[5.0, 3.0, 7.0]], dtype=float)
fractional = box.make_fractional(absolute)
absolute_restored = box.make_absolute(fractional)

{
    "points": points,
    "wrapped": wrapped,
    "images": images,
    "unwrapped_matches_original": bool(np.allclose(unwrapped, points)),
    "fractional": fractional,
    "absolute_restored_matches": bool(np.allclose(absolute_restored, absolute)),
}

## 5. Minimum-image distances

In a periodic system, the “closest” separation uses the minimum-image convention. Use:
`diff, r1, r2` for the minimum-image displacement vector and `dist, r1, r2` for the scalar distance.


In [ ]:
import numpy as np
import molpy as mp

box = mp.Box.cubic(10.0)

r1 = np.array([[1.0, 1.0, 1.0]], dtype=float)
r2 = np.array([[9.5, 9.5, 9.5]], dtype=float)

dr = box.diff(r1, r2)
d = box.dist(r1, r2)

points1 = np.array([[1.0, 1.0, 1.0], [2.0, 2.0, 2.0]], dtype=float)
points2 = np.array([[9.5, 9.5, 9.5], [8.0, 8.0, 8.0]], dtype=float)
distances = box.dist_all(points1, points2)

{
    "dr": dr,
    "distance": d,
    "dist_all_shape": distances.shape,
    "dist_all": distances,
}

## Summary

A `Box` is the cell geometry + PBC definition used for safe coordinate work., Construct common boxes with `cubic` / `orth` / `tric`, or provide a lattice matrix., Use `wrap` + fractional coordinates to keep periodic systems consistent., and Use `diff` / `dist` / `dist_all` for minimum-image distances..
